## Reasoning and Acting (ReAct)​

Agent to implement ReAct logic

In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "ls__632ae1a420dc4b46b2894dd749f33b2e"

os.environ["TAVILY_API_KEY"] = "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"

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://js-playground-01.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "43d936898d7645c3b3c1ebf294f04cd3"

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import AzureChatOpenAI

In [15]:
%pip install pypdf
%pip install pycryptodome
%pip install pymupdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 24.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/Underwriting Guidelines 319E 1.pdf")

data = loader.load()

data[0]

Document(page_content='BMO Insurance\nEffective Date:\nFebruary 26, 2024\n319E (2024/02/26)\nUnderwriting \nGuidelines\n', metadata={'source': 'data/Underwriting Guidelines 319E 1.pdf', 'file_path': 'data/Underwriting Guidelines 319E 1.pdf', 'page': 0, 'total_pages': 27, 'format': 'PDF 1.7', 'title': 'Underwriting Guidelines', 'author': 'BMO Insurance', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 19.1 (Macintosh)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20240126101951-05'00'", 'modDate': "D:20240126111742-05'00'", 'trapped': '', 'encryption': 'Standard V5 R6 256-bit AES'})

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

embeddings = AzureOpenAIEmbeddings(azure_deployment="ada-embedding", openai_api_version="2023-05-15")
vector = FAISS.from_documents(data, embeddings)
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "bmo_search",
    "Search for information about insurance underwriting. For any questions about insurance, you must use this tool!",
)

In [18]:
tools = [retriever_tool, TavilySearchResults(max_results=1)]

In [19]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [20]:
# Choose the LLM to use
llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt4-32k",
)

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [21]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
# agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
I need to find information about LangChain.
Action: tavily_search_results_json
Action Input: "what is LangChain"[{'url': 'https://www.langchain.com/', 'content': 'Join Discord\nContribute on GitHub\n5.3M\nMonthly Downloads\n67k\nGithub Stars\n1800+\nContributors\n34k+\nApps Powered\nReady to build?\nGo To Docs\nGo To Docs\nGo To Docs\nContact Sales\nContact Sales\nContact Sales\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\nJS/TS Docs\nSocial\nTwitter\nDiscord\nBlog\nLinkedIn\nYouTube\nTerms of Service\nSign up for our newsletter\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\nJS/TS Docs\nSocial\nTwitter\nDiscord\nBlog\nLinkedIn\nYouTube\nTerms of Service\nSign up for our newsletter\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\n

{'input': 'what is LangChain?',
 'output': 'LangChain is a tool for building language model applications. It provides standardized interfaces and integrations for interfacing with language models and application-specific data, making it easier to build and customize applications. It is designed to simplify the entire application lifecycle and supports the transition from prototypes to production. It is part of a broader ecosystem of tools and offers a range of features for application customization.'}

In [23]:
agent_executor.invoke(
    {"input": "What are the requirements for an Male applicant aged  70 for $9M ?"}
)



> Entering new AgentExecutor chain...
This question is about insurance underwriting, so I should use the bmo_search tool.
Action: bmo_search
Action Input: Requirements for a male applicant aged 70 for $9M insuranceReinstatements cont’d...
Life Insurance (Term) after 90 days from 
the lapse effective date for Non-Smoker 
– preferred or preferred rate classification
For amounts $100,000 to $500,000 and up to Age 65:
•	Long Form Health Certificate 167E
•	Urinalysis
For amounts greater than $500,000 or Age 65 and over:
•	Long Form Health Certificate 167E
•	Age and Amount Requirements
For all ages and amounts:
•	Long Form Health Certificate 167E
•	Age and Amount Requirements
Critical Illness Insurance within 90 days from 
the policy lapse effective date
For all ages and amounts:
•	Long Form Health Certificate 167E
Critical Illness Insurance after 90 days from 
the policy lapse effective date
•	Application for Life Insurance and Critical Illness Insurance 126E
•	Age and Amount Requirements

{'input': 'What are the requirements for an Male applicant aged  70 for $9M ?',
 'output': "For a male applicant aged 70 applying for $9M insurance, the requirements include a long form health certificate 167E, age and amount requirements, a tele-interview and vitals check, financial third party verification, and a motor vehicle report. The applicant's income and net worth need to be confirmed either by tax returns for the past 2 years or a letter from an accountant, and if for business purposes, financial statements for both the operating company and holding company for the last 3 years are required, along with a copy of the loan agreement and organizational charts. A cover letter outlining the purpose of the insurance, how the amount was determined, and any sales concepts used is also necessary."}